In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets
import torchvision
from torch.optim import Adam, SGD
from tqdm import tqdm
import csv
import cv2 as cv
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import random
import torch.nn.functional as F
import time
from transformers import SamModel, SamConfig, SamProcessor

from functools import partial
from typing import Any, Callable, Optional, Union

import torch
import torch.nn as nn
from torch import Tensor

# The following code is used and modified from the following github project:
# PyTorch Contributors. (2024). 
# ResNet model implementation. GitHub. https://github.com/pytorch/vision/blob/main/torchvision/models/resnet.py

# Portions of this code were generated with the assistance of ChatGPT (OpenAI, 2025) and subsequently modified by the author.
# OpenAI. (2025). ChatGPT (May 2025 version) [Large language model]. https://chat.openai.com

def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return nn.Conv2d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=dilation,
        groups=groups,
        bias=False,
        dilation=dilation,
    )


def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion: int = 1

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError("BasicBlock only supports groups=1 and base_width=64")
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.drop = nn.Dropout2d(0.4)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.drop(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition" https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion: int = 4

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.0)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):
    def __init__(
        self,
        block: type[Union[BasicBlock, Bottleneck]],
        layers: list[int],
        num_classes: int = 1,
        zero_init_residual: bool = False,
        groups: int = 1,
        width_per_group: int = 64,
        replace_stride_with_dilation: Optional[list[bool]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        #_log_api_usage_once(self)
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError(
                "replace_stride_with_dilation should be None "
                f"or a 3-element tuple, got {replace_stride_with_dilation}"
            )
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(Bottleneck, 64, layers[0])
        self.layer2 = self._make_layer(BasicBlock, 128, layers[1], stride=2, dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(BasicBlock, 256, layers[2], stride=2, dilate=True)
        self.layer4 = self._make_layer(Bottleneck, 512, layers[3], stride=2, dilate=True)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck) and m.bn3.weight is not None:
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, BasicBlock) and m.bn2.weight is not None:
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]

    def _make_layer(
        self,
        block: type[Union[BasicBlock, Bottleneck]],
        planes: int,
        blocks: int,
        stride: int = 1,
        dilate: bool = False,
    ) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(
            block(
                self.inplanes, planes, stride, downsample, self.groups, self.base_width, previous_dilation, norm_layer
            )
        )
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(
                block(
                    self.inplanes,
                    planes,
                    groups=self.groups,
                    base_width=self.base_width,
                    dilation=self.dilation,
                    norm_layer=norm_layer,
                )
            )

        return nn.Sequential(*layers)

    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)

In [ ]:
net50 = ResNet(Bottleneck, [2, 2, 2, 2], num_classes=1000)
#net50.load_state_dict(models.ResNet50_Weights.IMAGENET1K_V2.get_state_dict(progress=True, check_hash=True))

net50.fc = nn.Linear(net50.fc.in_features, 1)  # Change the output layer to match the number of classes

#net50.load_state_dict(torch.load('/content/dilate_resnet18_v3_all_model_classification_80.pth'))

# 2. Optimizer
optimizer = Adam([{'params':net50.layer4.parameters()},{'params': net50.fc.parameters()}], lr=1e-5, weight_decay=1e-5)
#optimizer = Adam(net50.parameters(), lr=1e-4, weight_decay=0)
#optimizer = SGD(model.parameters(),lr=0.001, momentum=0.9, weight_decay=1e-5)

# 3. MONAI Loss Function
loss_fn = nn.BCEWithLogitsLoss()
print(net50)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

net50.to(device)


In [ ]:
trainDirGlaucoma = "/content/glaucoma_classification/cropped_glaucoma/"
trainDirNormal = "/content/glaucoma_classification/cropped_normal/"

gl_im = []
nm_im = []

h = 256
w = 256

cnt1 = 0
cnt2 = 0
for path in os.listdir(trainDirGlaucoma):
    if path.endswith('.png'):
        img = cv.imread(cv.samples.findFile(trainDirGlaucoma + path))
        _,img_g,_ = cv.split(img)
        im = cv.cvtColor(img, cv.COLOR_BGR2HSV)
        _,_,img_v = cv.split(im)
        if img.shape == (h, w, 3):  # Check shape before adding
            #print(img.shape)
            cnt1 += 1
            gl_im += [img]

for path in os.listdir(trainDirNormal):
    if path.endswith('.png'):
        img = cv.imread(cv.samples.findFile(trainDirNormal + path))
        _,img_g,_ = cv.split(img)
        im = cv.cvtColor(img, cv.COLOR_BGR2HSV)
        _,_,img_v = cv.split(im)
        if img.shape == (h, w, 3):  # Check shape before adding
            #print(img.shape)
            cnt2 += 1
            nm_im += [img]

print(cnt1)
print(cnt2)

dataset_dict = {
    "image": [Image.fromarray(img) for img in gl_im] + [Image.fromarray(img) for img in nm_im],
    "label": [1 for _ in gl_im] + [0 for _ in nm_im],
}

In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(0.5),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet stats
                         std=[0.229, 0.224, 0.225])
])

In [ ]:


class MyDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images  # a list of PIL images
        self.labels = labels  # a list of 0s and 1s

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = train_transforms(self.images[idx])     # get the image at index `idx`
        label = self.labels[idx]         # get the label at index `idx`
        return {"image": image,"label": label,}

In [ ]:
dataset = MyDataset(dataset_dict["image"], dataset_dict["label"])
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

In [ ]:
#Training loop
num_epochs = 20

epoch_count = 81

start_time = time.perf_counter()
net50.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0

    for batch in tqdm(train_loader):
        images, labels = batch["image"].to(device), batch["label"].to(device)

        logits = net50(images)
        loss = loss_fn(logits, labels.unsqueeze(1).float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # Accuracy
        preds = (torch.sigmoid(logits) > 0.5).int()
        correct += (preds == labels.unsqueeze(1).int()).sum()
        total += labels.size(0)

    epoch_loss = running_loss / total
    acc = correct / total
    print(f"Epoch {epoch+1} | Loss: {epoch_loss:.4f} | Accuracy: {acc:.4f}")

    name = "dilate_resnet18_v4_all"

    write_header = not os.path.exists(name+"_model_classification.csv")

    with open(name+"_model_classification.csv", mode="a", newline="") as file:
      writer = csv.writer(file)
      if write_header:
          writer.writerow(["Time", "Epoch", "Loss", "Accuracy"])
      writer.writerow([time.perf_counter()-start_time,epoch_count+1, epoch_loss, acc])


    f1 = "/content/"+name+"_model_classification_"+str(epoch_count)+".pth"
    f0 = "/content/"+name+"_model_classification_"+str(epoch_count-1)+".pth"

    torch.save(net50.state_dict(), f1)

    if os.path.exists(f0):
        os.remove(f0)
        print("file: "+ name+"_model_classification_"+str(epoch_count-1)+".pth"+ " updated!")
    else:
       print("file: "+ name+"_model_classification_"+str(epoch_count)+".pth"+" created!")

    epoch += 1
    epoch_count+=1
